# Notebook 3:  Regression

<font color='red'>**Wichtig**:  
Dieses Notebook muss spätestens am Mittwoch den **22.12.2021** über die zugehörige Aufgabe auf Moodle abgegeben werden.</font>\
**Kompilieren** Sie vor der Abgabe das Notebook noch einmal **komplett** und speichern Sie es dann ab (_Kernel_ --> _Restart Kernel and Run All Cells_).\
Bevor Sie Ihr bearbeitetes Notebook hochladen, **benennen** Sie das Dokument bitte wie folgt:\
**_Nachname_Matrikelnr_Notebooknr.ipynb_**

Führen Sie nun nach und nach den unten stehenden Code aus.

## Import gebräuchlicher Bibliotheken

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Zeigt Plots direkt im Notebook an:
%matplotlib inline

## Beispiel Regression

Quellen:\
Müller, A.C. & Guido, S., Einführung in Machine Learning mit Python - Praxiswissen Data Science, O´Reilly (2017)\
https://github.com/amueller/introduction_to_ml_with_python

Als Beispiel für eine Regressionsaufgabe werden wir vorerst auf einem generierten Datensatz arbeiten. Zu Zwecken der Visualisierung besteht dieser erste Beispieldatensatz aus nur einem Feature X und den Labels y.

In [ ]:
# Datensatz generieren
def make_wave(n_samples=100):
    rnd = np.random.RandomState(42)
    x = rnd.uniform(-3, 3, size=n_samples)
    y_no_noise = (np.sin(4 * x) + x)
    y = (y_no_noise + rnd.normal(size=len(x))) / 2
    return x.reshape(-1, 1), y

X, y = make_wave(n_samples=60)
plt.plot(X, y, 'o')
plt.ylim(-3, 3)
plt.xlabel("Feature")
plt.ylabel("Labels")

### Lineare Regression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Trein-Test-Split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Trainiere lineare Regression
lr = LinearRegression().fit(X_train, y_train)

# Der y-Achsenabschnitt (intercept) wird in der Implementierung gesondert betrachtet
print("Gewicht w[0]: %f  y-Achsenabschnitt b: %f" % (lr.coef_[0], lr.intercept_))

**Bemerkung**:\
Scikit Learn speichert alles was aus den Trainingsdaten berechnet wurde mit einem nachgestellten Unterstrich, um es von Parametern zu unterscheiden, welche vom User gesetzt wurden.

In [ ]:
# Plot
line = np.linspace(-3, 3, 100).reshape(-1, 1)
plt.figure(figsize=(8, 8))
plt.plot(line, lr.predict(line))
plt.plot(X, y, 'o')
ax = plt.gca()
ax.spines['left'].set_position('center')
ax.spines['right'].set_color('none')
ax.spines['bottom'].set_position('center')
ax.spines['top'].set_color('none')
ax.set_ylim(-3, 3)
#ax.set_xlabel("Feature")
#ax.set_ylabel("Label")
ax.legend(["Modell", "Trainingsdaten"], loc="best")
ax.grid(True)
ax.set_aspect('equal')

Schauen wir uns nun die Performance auf den Trainings- und Testdaten an:

In [ ]:
print("Trainingsscore: {:.2f}".format(lr.score(X_train, y_train)))
print("Testscore: {:.2f}".format(lr.score(X_test, y_test)))

Ein Score von 1 ist das beste Ergebnis und ein Score von 0 das Schlechteste.

Ein Testscore von 0.69 ist nicht so gut. Aber man kann sehen, dass Trainings- und Testscore nahe beieinander sind, d.h. das wir höchstwahrscheinlich eher Underfitting als Overfitting haben. (Die Gefahr für Overfitting ist bei eindimensionalen Datensätzen (d.h. nur ein Feature) verhältnismäßig gering, bei mehrdimensionalen Datensätzen (d.h. viele Featuures) ist die Gefahr viel größer.)

Als Alternative zu diesem einfachen Datensatz wollen wir uns einen komplexeren Datensatz von Diabetes-Patienten ansehen:

In [ ]:
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()

# Features & labels
X = diabetes.data
y = diabetes.target

print("Dimension der Daten:", X.shape)

Informationen zum Diabetes Datensatz sind hier zu finden:\
https://scikit-learn.org/stable/datasets/toy_dataset.html#diabetes-dataset \
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html

Der Datensatz enthält 10 verschiedene gemessene Werte von 442 Diabetes-Patienten. Die Labels/Zielvariable enthält ein quantitatives Maß des Krankheitsverlaufs über ein Jahr.
Um den Datensatz komplexer zu machen, fügen wir das Produkt je zweier Spalten als neue Features hinzu (dies entspricht quadratischer Regression):

In [ ]:
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures

# Feature Engineering:
X = MinMaxScaler().fit_transform(X)     # Skaliere Features zwischen 0 und 1
X = PolynomialFeatures(degree=2, include_bias=False).fit_transform(X)     # Produkt je zweier Spalten hinzufügen

print("Dimension der Daten:", X.shape)

Nun teilen wir wieder unsere Daten auf und trainieren unser Regressionsmodell:

In [ ]:
# Train-Test-Split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Trainiere lineare Regression
lr = LinearRegression().fit(X_train, y_train)

# Training & Test Score
print("Trainingsscore: {:.2f}".format(lr.score(X_train, y_train)))
print("Testscore: {:.2f}".format(lr.score(X_test, y_test)))

Der Testscore ist sehr schlecht, der Trainigsscore ist auch nicht gut, aber deutlich besser. Ein großer Unterschied zwischen Training- und Testscore ist ein Indikator für Overfitting. Um Overfitting zu vermeiden schauen wir uns nun regularisierte Regressionsmodelle an.

### Ridge Regression

Schauen wir uns im Vergleich die Performance der Ridge Regression an:

In [ ]:
from sklearn.linear_model import Ridge

# Trainiere Ridge Regression
ridge = Ridge().fit(X_train, y_train)

# Training & Test Score
print("Trainingsscore: {:.2f}".format(ridge.score(X_train, y_train)))
print("Testscore: {:.2f}".format(ridge.score(X_test, y_test)))

Man kann sehen, dass der Wert des Trainingsscores niedriger und der des Testscores höher ist. Dies ist zu erwarten, da ein regularisiertes Modell besser generalisieren sollte und nicht so sehr overfittet.

Ridge verwendet ohne eine Einstellung des Regularisierungsparameters, der hier alpha heißt, alpha=1.
Erhöht man den Regularisierungsparameter alpha, so wird die Performance auf den Trainingsdaten schlechter aber der Testscore wird besser, d.h. das Modell generalisiert besser:

In [ ]:
# Trainiere Ridge Regression
ridge = Ridge(alpha=10).fit(X_train, y_train)

# Training & Test Score
print("Trainingsscore: {:.2f}".format(ridge.score(X_train, y_train)))
print("Testscore: {:.2f}".format(ridge.score(X_test, y_test)))

Verkleinern wir alpha, so gehen wir mehr in Richtung des unregularisierten Modells. Testen Sie verschiedene Regularisierungsparameter aus:

In [ ]:
# Trainieren Sie Ridge Regression mit verschiedenen alphas


### Lasso Regression

Alternativ können wir Lasso Regression betrachten.

In [ ]:
from sklearn.linear_model import Lasso

# Trainiere Lasso Regression
lasso = Lasso().fit(X_train, y_train)

# Training & Test Score
print("Trainingscore: {:.2f}".format(lasso.score(X_train, y_train)))
print("Testscore: {:.2f}".format(lasso.score(X_test, y_test)))
print("Anzahl benutzter Features:", np.sum(lasso.coef_ != 0))

Lasso performt schlechter auf den Trainings- und Testdaten. Wahrscheinlich findet hier Underfitting statt, es werden auch nur 9 von 65 Features verwendet.

Auch bei Lasso ist der Standardregularisierungsparameter alpha=1. Schauen wir uns an, was passiert, wenn wir den Regularisierungsparameter alpha veringern. Wenn wir das machen, müssen wir auch den Standardwert von max_iter=1000 (maximale Anzahl der Iterationen) erhöhen:

In [ ]:
# Trainiere Lasso Regression
lasso001 = Lasso(alpha=0.5, max_iter=5000).fit(X_train, y_train)

# Training & Test Score
print("Trainingscore: {:.2f}".format(lasso001.score(X_train, y_train)))
print("Testscore: {:.2f}".format(lasso001.score(X_test, y_test)))
print("Anzahl benutzter Features:", np.sum(lasso001.coef_ != 0))

Testen Sie auch für die Lasso Regression verschiedene Einstellungen:

In [ ]:
# Trainieren Sie Lasso Regression mit verschiedenen alphas


Der Datensatz ist nur zum Ausprobieren und nicht sehr ergiebig. Aber es reicht um ein grobes Verständnis der Methoden zu bekommen.

# Anwendung: California Housing Datensatz

Für die Übungsaufgaben nutzen wir wieder den California Housing Datensatz.\
Quellen:\
http://lib.stat.cmu.edu/datasets/houses.zip \
https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset \
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html

**Erinnerung**:

- **MedHouseVal:**  Median-Hauswert --> Label

Die Features sind die folgenden:

- **MedInc:**      Median-Einkommen
- **HouseAge:**    Median-Hausalter
- **AveRooms:**    Mittlere Zimmerzahl
- **AveBedrms:**   Mittlere Schlafzimmerzahl  
- **Population:**  Bevölkerungszahl
- **AveOccup:**    Mittlere Bewohnerzahl der Häuser
- **Latitude:**    Breitengrad
- **Longitude:**   Längengrad 

Laden des California Housing Datensatzes:

In [ ]:
from sklearn.datasets import fetch_california_housing

(X, y) = fetch_california_housing(return_X_y=True)

print("Dimension der Daten:", X.shape)

## Aufgabe 1 (10 Punkte): 

Trainieren Sie nichtregularisierte Regressionsmodelle auf dem California Housing Datensatz:

a) Trainieren Sie ein einfaches lineares Regressionsmodell __nur__ mit dem Feature Median-Hausalter. Plotten und evaluieren Sie das Modell. Was fällt Ihnen auf? Was könnte man verbessern?

b) Trainieren Sie nun ein einfaches lineares Regressionsmodell basierend auf allen Features. Welche informationen erhalten Sie aus Trainings- & Testscore?

c) Trainieren ein polynomiales Regressionsmodell vom Grad 2 und Grad 3 basierend auf allen Features. Was fällt Ihnen auf?\
**Bemerkung**:\
Bei polynomialer Regression mit mehr als einem Feature werden nicht nur die Potenzen einzelner Features sondern auch die jeweiligen Produkte unterschiedlicher Features betrachtet.\
Eine Hilfestellung zur Implementierung der polynomialen Regression können Sie z.B. hier finden:\
https://scikit-learn.org/stable/modules/linear_model.html#polynomial-regression-extending-linear-models-with-basis-functions

## Aufgabe 2 (10 Punkte):

Trainieren Sie regularisierte Modelle auf dem California Housing Datensatz:

a) Trainieren Sie ein Ridge Regression Modell und evaluieren Sie es. Wie erhalten Sie ein möglichst gutes und sinnvolles Modell? (Regularisierungsparameter? Lineare oder polynomiale Regression?) Was fällt Ihnen auf?

b) Trainieren Sie ein Lasso Regression Modell und evaluieren Sie es. Wie erhalten Sie ein möglichst gutes und sinnvolles Modell? (Regularisierungsparameter? Lineare oder polynomiale Regression?) Was fällt Ihnen auf?

c) Vergleichen Sie die verschiedenen Regressionsansätze, d.h. nichtregularisierte Regression (Methode der kleinsten Quadrate), Ridge Regression und Lasso Regression.

## Bonusaufgabe (10 Extrapunkte): 

Implementieren Sie die Ridge Regression von Hand. 

- Schreiben Sie eine Funktion, die Ridge Regression auf eingegebene Daten (Feature Matrix X und Labels y) anwendet (ohne zu Hilfenahme des Scikit Learn Pakets).
- Wenden Sie Ridge Regression auf den generierten Wave-Datensatz an (siehe oben).
- Probieren Sie die Kombination von polynomialer Regression (hierfür darf sklearn.preprocessing.PolynomialFeatures verwendet werden) mit Ridge Regression basierend auf dem oben generierten Wave-Datensatz.
- Wie erhalten Sie die besten bzw. gute Ergebnisse?